In [1]:
import httpx
import time
import json
from bs4 import BeautifulSoup

In [2]:

scrapedData = {}

visitedURLs = []

visiteddomains=[]

rss_feeds=[]

emails=[]

In [3]:
with open('scrapedDatabase/scrapedData.json', 'r') as file:
    scrapedData = json.load(file)

In [4]:
with open('scrapedDatabase/visitedURLs.json', 'r') as file:
    visitedURLs = json.load(file)

In [5]:
import RAKE

text_en = (
    'Compatibility of systems of linear constraints over the set of '
    'natural numbers. Criteria of compatibility of a system of linear '
    'Diophantine equations, strict inequations, and nonstrict inequations '
    'are considered. Upper bounds for components of a minimal set of '
    'solutions and algorithms of construction of minimal generating sets '
    'of solutions for all types of systems are given. These criteria and '
    'the corresponding algorithms for constructing a minimal supporting '
    'set of solutions can be used in solving all the considered types of '
    'systems and systems of mixed types.'
)

rake = RAKE.Rake('SmartStoplist.txt')


def srt(lst):
    lst.sort(key=lambda x:x[1])
    return lst

print(srt(rake.run(text_en))[-10:])

def get_top_kwds(text):
    return srt(rake.run(text.replace('\n',' ')))

[('mixed types', 3.666666666666667), ('natural numbers', 4.0), ('strict inequations', 4.0), ('nonstrict inequations', 4.0), ('upper bounds', 4.0), ('linear constraints', 4.5), ('minimal set', 4.666666666666666), ('minimal supporting set', 7.666666666666666), ('linear diophantine equations', 8.5), ('minimal generating sets', 8.666666666666666)]


In [6]:
strictness=3.5
def find_topic(sentence):
    '''
    # Process the sentence
    doc = nlp(sentence)
    
    # Extract the most relevant noun or noun chunk
    for chunk in doc.noun_chunks:
        return chunk.text
        '''
    top_kwds= get_top_kwds(sentence)
    return top_kwds[-len(top_kwds)//10:]


In [7]:
from bs4.element import Comment


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [8]:

def find_document_topic(soup):
    paragraphs = soup.findAll(string=True)
    visible_texts = filter(tag_visible, paragraphs)  

    sentence = u" ".join(t.strip() for t in visible_texts)

    topic = find_topic(sentence)

    return topic


In [9]:

def extract_links(soup):
    # Find all the anchor tags with href attributes
    links = [a['href'] for a in soup.find_all('a', href=True)]

    return links


In [10]:
def addURL_kwd(kwd,url,queries={},fragment=''):
    if kwd not in scrapedData:
        scrapedData[kwd]={
            url: queries
        }
        return
    if url not in scrapedData[kwd]:
        scrapedData[kwd][url]=queries
    scrapedData[kwd][url].update(queries)
    return

def deconstruct_url(url):
    #assuming URL is not malformed.
    scheme,*url=url.split('://')
    url='://'.join(url)
    url,*fragment=url.split('#')
    fragment=(fragment+[''])[0]
    url=url.replace('&','?').strip('?')
    auth_n_path,*strqueries=url.split('?')
    queries=dict([tuple(query.split('=')) for query in strqueries])
    if auth_n_path[-1]=='/':
        auth_n_path=auth_n_path[:-1]
    auth,*path=auth_n_path.split('/')
    return scheme,auth,path,queries,fragment

def reconstruct_url(scheme='',auth='',path=[],queries={},fragment=''):
    return f'{scheme}://{auth}/'+ '/'.join(path)+ '?'*(len(queries)>0)+ '&'.join([f'{a}={b}' for a,b in queries.items()])+ '#'*(fragment!='')+fragment

def hasScheme(url):
    return '://' in url


def contextify_url(previous_url,current_url):
    if current_url=='':
        return previous_url
    if hasScheme(current_url):
        return current_url
    pscheme,pauth,ppath,pqueries,pfragment = deconstruct_url(previous_url)

    current_url,*cfragment=current_url.split('#')
    cfragment=(cfragment+[''])[0]
    current_url=current_url.replace('&','?').strip('?')
    current_url,*cstrqueries=current_url.split('?')
    cqueries=dict([tuple(query.split('=')) for query in cstrqueries])
    cauth_or_pathswitcher,*cpath=current_url.split('/')

    if '.' in cauth_or_pathswitcher: # balls.com/blah?he=he#haha
        return reconstruct_url(pscheme,cauth_or_pathswitcher,cpath,cqueries,cfragment)
    cpath=[cauth_or_pathswitcher]+cpath
    if cauth_or_pathswitcher=='':   # /blah?he=he#haha
        return reconstruct_url(pscheme,pauth,cpath,cqueries,cfragment)
    return reconstruct_url(pscheme,pauth,ppath+cpath,cqueries,cfragment)
    

In [11]:
ex_url='https://balls.com'
a=deconstruct_url(ex_url)
print(a)
b=reconstruct_url(*a)
print(b)

('https', 'balls.com', [], {}, '')
https://balls.com/


In [12]:
ex_goto='secret_location/tower'
contextify_url(ex_url,ex_goto)

'https://balls.com/secret_location/tower'

In [13]:

scrapestack=[]


In [14]:
pagehashes=[]

In [15]:
blacklist=[
    '.pdf',
    '.jpeg'
]
def scrape(url):
    global scrapestack
    if url in visitedURLs:
        print('URL was already finished.')
        return
    
    if 'mailto' in url and url not in emails:
        emails.append(url)
        print('That was an email!')
        return
    
    if 'feed.xml' in url and url not in rss_feeds:
        rss_feeds.append(url)
        print('What was an RSS FEED!')
        return

    visitedURLs.append(url)

    with open('scrapedDatabase/visitedURLs.json', 'w') as file:
        json.dump(visitedURLs, file, indent=4)

    
    scheme,auth,path,queries,fragment=deconstruct_url(url)
    print(scheme,auth,path,queries,fragment)

    if (dom:=f'{scheme}://{auth}') not in visiteddomains:
        visiteddomains.append(dom)
    
    try:
        response = httpx.get(url, timeout=2)
    except Exception as e:
        print('Error encountered in response for url '+url+': '+str(e))
        return
    try:
        response.raise_for_status() # Raise an exception for HTTP errors
    except Exception as e:
        print(f"Error fetching the URL: {e}")
        for i in str(e).replace('"',' ').replace('\'',' ').split(' ')[1:]:
            if hasScheme(i):
                scrapestack.append(i)
        return
    if (hsh:=hash(response.content)) in pagehashes:
        print('This page has been encountered before!')
        return
    pagehashes.append(hsh)
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    links = extract_links(soup)

    topics = find_document_topic(soup)
    try:
        topics+=(''.join(map(lambda x: x if x.isalnum() or x==' ' else '', soup.title.text))).split(' ')
    except:
        print("Document had no title.")

    for topic in topics:
        if len(topic)==2:
            topic=topic[0].lower().replace('-',' ')

            if ('  ' not in ''.join([a if a.isalpha() else ' ' for a in topic])):
                print(topic)
                for t in topic.split(' '):
                    addURL_kwd(t,reconstruct_url(scheme,auth,path),queries=queries)

    for link in links:
        try:
            scrapestack.append(contextify_url(url,link))
        except:
            print(f'There was a problem while contextifying link {link} with url {url}')


In [16]:

startingURL = 'https://neocities.org/'
scrape(startingURL)

https neocities.org [] {} 
fast site performance
favorite desktop editor
command line tool
drag-n-drop
caching anycast cdn
routinely out-perform
pricey cloud services
static web hosting
fast static hosting
in-browser html editor
offer free static web hosting
great in-browser html editor


In [17]:
def save():
    with open('scrapedDatabase/scrapedData.json', 'w') as file:
        json.dump(scrapedData, file, indent=4)

In [18]:
for i in range(50000):
    print(f'({i}/{len(scrapestack)} remaining!)')
    #try:
    print(scrapestack[0])
    try:
        scrape(scrapestack.pop(0))
    except:
        print('Big error')
    print('\n')
    if i%100==1:
        print("Saving...")
        save()
        print("Saved!")
    #except Exception as e:
    #    print(f'Failed: {e}')

(0/48 remaining!)
https://neocities.org/#!
https neocities.org [] {} !
fast site performance
favorite desktop editor
command line tool
drag-n-drop
caching anycast cdn
routinely out-perform
pricey cloud services
static web hosting
fast static hosting
in-browser html editor
offer free static web hosting
great in-browser html editor


(1/95 remaining!)
https://neocities.org//
https neocities.org [''] {} 
fast site performance
favorite desktop editor
command line tool
drag-n-drop
caching anycast cdn
routinely out-perform
pricey cloud services
static web hosting
fast static hosting
in-browser html editor
offer free static web hosting
great in-browser html editor


Saving...
Saved!
(2/142 remaining!)
https://neocities.org//browse
https neocities.org ['', 'browse'] {} 


(3/1092 remaining!)
https://neocities.org//activity
https neocities.org ['', 'activity'] {} 
Error encountered in response for url https://neocities.org//activity: The read operation timed out


(4/1091 remaining!)
https://ne

In [ ]:
visiteddomains

['https://neocities.org',
 'https://distantskies.neocities.org',
 'https://kekkonen.neocities.org',
 'https://gifypet.neocities.org',
 'https://civilservant.io',
 'https://that1archive.neocities.org',
 'https://lainzine.org',
 'https://github.com',
 'http://www.wired.com',
 'http://www.fastcodesign.com',
 'http://motherboard.vice.com',
 'http://arstechnica.com',
 'http://web.appstorm.net',
 'https://twitter.com',
 'https://www.facebook.com',
 'http://status.neocitiesops.net']

In [ ]:
emails

[]

In [ ]:
rss_feeds

[]